# agex: A Demo of Runtime Interoperability

Welcome to the `agex` demo! This notebook showcases the core philosophy of the framework: enabling AI agents to work with real, rich Python objects, just like a human developer.

**The Story:** We, as the developer, will act as the orchestrator. Our goal is to create a sales trend visualization. We will use two specialist AI agents:
1. A `DatabaseExpert` that can query a database and return a `pandas.DataFrame`.
2. A `VisualizationExpert` that can take a `DataFrame` and create a `plotly.Figure`.

Notice what we **don't** have to do: no writing JSON schemas, no saving data to files to pass between steps, and no context window explosions. Let's begin.


### Step 1: Setup and Environment

First, let's import our libraries and create an in-memory SQLite database populated with some realistic, seasonal sales data. The `create_in_memory_db` function returns a live `sqlite3.Connection` object.


In [ ]:
import pandas as pd
import plotly.express as px
import sqlite3
from IPython.display import display
from agex import Agent, Versioned, events
from agex.helpers import register_pandas 
from agex.llm import DummyLLMClient
from plotly.graph_objects import Figure
from examples.db_primer import PRIMER

# Our helpers for demo setup and canned responses
from examples.notebook.helper import (
    create_in_memory_db, 
    DB_QUERY_RESPONSE, 
    VISUALIZATION_RESPONSE
)

db_connection = create_in_memory_db()

# create persistent state for the agents & their computational workspaces
state = Versioned()

### Step 2: Define and Use the DatabaseExpert

Next, we'll define our `DatabaseExpert`. We'll give it access to the live `db_connection` object and the `pandas` library. 

To make this demo fast and reproducible, we'll use a `DummyLLMClient` with pre-defined responses. This lets us focus on `agex`'s unique capabilities without the unpredictability of live LLM calls.


In [2]:
db_expert = Agent(
    name="db_expert", 
    primer=PRIMER, # coach the agent on how to use the db
)
db_expert.llm_client = DummyLLMClient(responses=[DB_QUERY_RESPONSE])

# Give the agent access to the live database connection
db_expert.module(
    db_connection,
    name="db",
    include=["execute", "executemany", "commit"],
)

# register the Cursor class for gathering results
db_expert.cls(sqlite3.Cursor, include=["fetchone", "fetchall", "description"])

# use the pre-existing helper to register pandas
register_pandas(db_expert)

# define a task for our agent, the fn signature acts as a contract
@db_expert.task
def get_data(prompt: str) -> pd.DataFrame: # type: ignore[return-value]
    """Runs a SQL query and returns the result as a pandas DataFrame."""
    pass

# now we ask the agent to get the sales data for umbrellas.
sales_df = get_data(
    prompt="Find all available umbrella sales by day",
    state=state,
)

### Step 3: Inspect the First Artifact

This is the first **'wow' moment**. The `get_data` call didn't return a string or JSON. It returned a live, rich `pandas.DataFrame` object directly into our session. The agent came to the data, performed its work, and handed back a real object.

Let's prove it by displaying the DataFrame:


In [3]:
display(sales_df)

,date,product_name,quantity_sold,price_per_unit
0,2022-01-01,Umbrellas,4,15
1,2022-01-02,Umbrellas,9,15
2,2022-01-03,Umbrellas,3,15
3,2022-01-04,Umbrellas,3,15
4,2022-01-05,Umbrellas,13,15
...,...,...,...,...
725,2023-12-27,Umbrellas,0,15
726,2023-12-28,Umbrellas,4,15
727,2023-12-29,Umbrellas,3,15
728,2023-12-30,Umbrellas,0,15


### Step 4: Define and Use the VisualizationExpert

Now we'll define our second specialist. Its job is to take a DataFrame and create a plot. We will pass the `sales_df` object we just received directly to this new agent.


In [4]:
viz_expert = Agent(
    name="viz_expert",
    primer="You are an expert at creating plots."
)
viz_expert.llm_client = DummyLLMClient(responses=[VISUALIZATION_RESPONSE])

# Give the agent access to the plotly express library
viz_expert.module(px)
register_pandas(viz_expert)

@viz_expert.task
def plot_dataframe(prompt: str, df: pd.DataFrame) -> Figure: # type: ignore[return-value]
    """Create a plot from the given data according to the prompt."""
    pass

# Let's ask the agent to plot the DataFrame we got from the first agent.
print("Passing the DataFrame to the Visualization Expert...")
sales_plot = plot_dataframe(
    "Please show me overlayed yearly sales cycles",
    sales_df,
    state=state
)

Passing the DataFrame to the Visualization Expert...


### Step 5: Inspect the Final Artifact

And here is the second **'wow' moment**. We passed a `DataFrame` object from one agent to another, and the second agent returned a fully interactive `plotly.Figure` object. 

Let's display it:


In [5]:
display(sales_plot)

Curious to see the entire flow from start to finish?

We can replay all the internal events collected from our agents during those tasks thanks to our persistent state!


In [6]:
for evt in events(state):
    display(evt)

TaskStartEvent(TaskStartEvent[db_expert] @ 12:33:25)

ActionEvent(ActionEvent[db_expert] @ 12:33:25)

,date,product_name,quantity_sold,price_per_unit
0,2022-01-01,Umbrellas,4,15
1,2022-01-02,Umbrellas,9,15
2,2022-01-03,Umbrellas,3,15
3,2022-01-04,Umbrellas,3,15
4,2022-01-05,Umbrellas,13,15
...,...,...,...,...
725,2023-12-27,Umbrellas,0,15
726,2023-12-28,Umbrellas,4,15
727,2023-12-29,Umbrellas,3,15
728,2023-12-30,Umbrellas,0,15


,date,product_name,quantity_sold,price_per_unit
0,2022-01-01,Umbrellas,4,15
1,2022-01-02,Umbrellas,9,15
2,2022-01-03,Umbrellas,3,15
3,2022-01-04,Umbrellas,3,15
4,2022-01-05,Umbrellas,13,15
...,...,...,...,...
725,2023-12-27,Umbrellas,0,15
726,2023-12-28,Umbrellas,4,15
727,2023-12-29,Umbrellas,3,15
728,2023-12-30,Umbrellas,0,15


ActionEvent(ActionEvent[viz_expert] @ 12:33:25)

SuccessEvent(SuccessEvent[viz_expert] @ 12:33:25)

### Conclusion: The `agex` Difference

We just orchestrated a complex workflow involving a database, a large dataset, and a visualization library, all using natural language prompts.

Crucially, we leveraged the powerful abstractions we *already* use as developers (`pandas.DataFrame`, `plotly.Figure`) instead of being forced into a restrictive tool-calling paradigm. The agents worked with our existing code and objects natively.

This is the core advantage of `agex`: it respects your code and enables a more powerful, flexible, and Pythonic way to build agentic systems.

---

**Want to see what the agents actually generated?** The responses used in this demo are stored in `examples/notebook/helper.py` as `DB_QUERY_RESPONSE` and `VISUALIZATION_RESPONSE`. They are real LLM outputs, captured for reproducibility.
